In [99]:
import tensorflow as tf

In [100]:
tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [101]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [102]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [103]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [104]:
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

In [105]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [106]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [107]:
input_shape = X_train.shape[1:]

In [108]:
import keras
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [109]:
model.compile(loss=huber_fn, optimizer="nadam")

In [110]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5960 - val_loss: 0.2346
Epoch 2/10
363/363 [==============================] - 0s 918us/step - loss: 0.2154 - val_loss: 0.1993
Epoch 3/10
363/363 [==============================] - 0s 904us/step - loss: 0.2032 - val_loss: 0.2029
Epoch 4/10
363/363 [==============================] - 0s 889us/step - loss: 0.1985 - val_loss: 0.1896
Epoch 5/10
363/363 [==============================] - 0s 972us/step - loss: 0.1954 - val_loss: 0.1857
Epoch 6/10
363/363 [==============================] - 0s 916us/step - loss: 0.1915 - val_loss: 0.2032
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1909 - val_loss: 0.1882
Epoch 8/10
363/363 [==============================] - 0s 885us/step - loss: 0.1883 - val_loss: 0.1813
Epoch 9/10
363/363 [==============================] - 0s 898us/step - loss: 0.1863 - val_loss: 0.1763
Epoch 10/10
363/363 [==============================] - 0s 883us/step - loss: 0.1841 - 

In [111]:
import keras
import numpy as np
class LayerNormalizationn(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
    def build(self, input_shape):
        self.alpha = self.add_weight(
            shape=(input_shape[-1:]), name="alpha", initializer="ones"
        )
        self.beta = self.add_weight(shape=(input_shape), name="beta", initializer="zeros")
    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        std_deviation = tf.sqrt(variance + self.eps)
        return self.alpha * (X - mean)/(std_deviation) + self.beta

X = X_train.astype(np.float32)

print(X_train.shape)

(11610, 8)


In [112]:
custom_layer_norm = LayerNormalizationn()

keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=3.9357733e-08>

In [113]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [118]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

X_train_full = X_train_full.astype(np.float32) / 255.0

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test.astype(np.float32) / 255.0


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
def random_batch(X, y, batch_size):
    idx = np.random.randint(0, len(X), size=batch_size)
    return X[idx], y[idx]
batch_size = 32
n_steps = len(X_train) // batch_size
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

optimizer = keras.optimizers.Nadam(learning_rate=0.01)
n_epochs = 5

In [119]:
from tqdm.auto import tqdm  # Usando auto para adaptar ao ambiente (notebook, terminal, etc)
from collections import OrderedDict

# Barra de progresso para os epochs
for epoch in tqdm(range(1, n_epochs + 1), desc="All epochs", dynamic_ncols=True):
    mean_loss.reset_states()

    # Barra de progresso para os steps dentro de cada epoch
    step_bar = tqdm(range(1, n_steps + 1), desc=f"Epoch {epoch}/{n_epochs}", leave=False, dynamic_ncols=True)
    for step in step_bar:
        X_batch, y_batch = random_batch(X_train, y_train, batch_size=batch_size)

        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        status = OrderedDict()
        status["loss"] = mean_loss.result().numpy()

        for metric in metrics:
            metric(y_batch, y_pred)
            status[metric.name] = metric.result().numpy()

        step_bar.set_postfix(status)

    y_pred = model(X_valid)
    val_loss = np.mean(loss_fn(y_valid, y_pred))
    val_accuracy = np.mean(keras.metrics.sparse_categorical_accuracy(
        tf.constant(y_valid, dtype=np.float32), y_pred))

    tqdm.write(f"Epoch {epoch}/{n_epochs} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")

    for metric in [mean_loss] + metrics:
        metric.reset_states()


All epochs:   0%|                                                                                | 0/5 [00:00<…

Epoch 1/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 1/5 - val_loss: 0.4300 - val_accuracy: 0.8496


Epoch 2/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 2/5 - val_loss: 0.4251 - val_accuracy: 0.8506


...

Epoch 3/5 - val_loss: 0.4150 - val_accuracy: 0.8580


Epoch 4/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 4/5 - val_loss: 0.3926 - val_accuracy: 0.8630


Epoch 5/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 5/5 - val_loss: 0.3882 - val_accuracy: 0.8730


In [128]:
keras.backend.clear_session()
lower_layers = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(100, activation="relu")
])

upper_layers = keras.models.Sequential([
    keras.layers.Dense(10, activation="softmax")
])

model = keras.models.Sequential([
    lower_layers, upper_layers
])

lower_optimizer = keras.optimizers.SGD(learning_rate=0.0001)
upper_optimizer = keras.optimizers.Nadam(learning_rate=1e-3)

In [130]:
n_steps = len(X_train) // batch_size
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

n_epochs = 5
for epoch in tqdm(range(1, n_epochs + 1), desc="All epochs", dynamic_ncols=True):
    mean_loss.reset_states()

    step_bar = tqdm(range(1, n_steps + 1), desc=f"Epoch {epoch}/{n_epochs}", leave=False, dynamic_ncols=True)
    for step in step_bar:
        X_batch, y_batch = random_batch(X_train, y_train, batch_size=batch_size)

        with tf.GradientTape(persistent=True) as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        for layers, optimizers in ((lower_layers, lower_optimizer), (upper_layers, upper_optimizer)):
            gradients = tape.gradient(loss, layers.trainable_variables)
            optimizer.apply_gradients(zip(gradients, layers.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        status = OrderedDict()
        status["loss"] = mean_loss.result().numpy()

        for metric in metrics:
            metric(y_batch, y_pred)
            status[metric.name] = metric.result().numpy()

        step_bar.set_postfix(status)

    y_pred = model(X_valid)
    val_loss = np.mean(loss_fn(y_valid, y_pred))
    val_accuracy = np.mean(keras.metrics.sparse_categorical_accuracy(
        tf.constant(y_valid, dtype=np.float32), y_pred))

    tqdm.write(f"Epoch {epoch}/{n_epochs} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")

    for metric in [mean_loss] + metrics:
        metric.reset_states()


All epochs:   0%|                                                                                | 0/5 [00:00<…

Epoch 1/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 1/5 - val_loss: 0.5786 - val_accuracy: 0.7888


Epoch 2/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 2/5 - val_loss: 0.5190 - val_accuracy: 0.8112


...

Epoch 3/5 - val_loss: 0.5595 - val_accuracy: 0.8142


Epoch 4/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 4/5 - val_loss: 0.5170 - val_accuracy: 0.8306


Epoch 5/5:   0%|                                                                              | 0/1718 [00:00<…

Epoch 5/5 - val_loss: 0.4843 - val_accuracy: 0.8368
